In [1]:
# This file is the early version to do pre-processing to get bug features and use Dirichlet Process Gaussian Mixture Model to cluster bugs. The
# performance is poor so I abandon it.
import MySQLdb
import pandas
import re
import itertools
from nltk.corpus import stopwords
from nltk import PorterStemmer
from bs4 import BeautifulSoup
#LCAL_DATABASE_HOST = "10.117.8.41"
#LOCAL_DATABASE_PORT = 3306
#LOCAL_DATABASE_USER = "root"
#LOCAL_DATABASE_PW = "vmware"
#LOCAL_DATABASE_DATABASE = "bugdata"
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur =conn.cursor()

#sql = '''SELECT bug_id 
#FROM bugs
#WHERE assigned_to in (
#SELECT userid 
#FROM profiles 
#WHERE login_name in ('hfu','letian','vbhakta','weili','nmukuri','zhoum','hxie','shiyaoy',
#'shanpeic','souravk','vaibhavk','fangchiw','gengshengl'))
#LIMIT 0, 1000'''

sql = '''SELECT bug_id 
FROM bugs,profiles 
WHERE bugs.assigned_to=profiles.userid and profiles.login_name in ('hfu','letian','vbhakta','weili','nmukuri','zhoum','hxie','shiyaoy','shanpeic','souravk','vaibhavk','fangchiw','gengshengl')
LIMIT 0, 1000'''

cur.execute(sql)
bug_id = cur.fetchall()
bug_id = [str(item[0]) for item in bug_id]
print len(bug_id)


1000


In [2]:
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugdata')
cur =conn.cursor()
sql_sd = '''SELECT bug_id, short_desc
FROM bugs
WHERE bug_id IN (%s)
ORDER BY bug_id''' %  ','.join(bug_id)

df_sd = pandas.io.sql.read_sql(sql_sd, conn)

print df_sd.dtypes

bug_id         int64
short_desc    object
dtype: object


In [3]:
sql_ld = '''SELECT bug_id, group_concat('\n', longdescs.thetext) description
FROM longdescs
WHERE bug_id IN (%s)
GROUP BY bug_id
ORDER BY bug_id''' % ','.join(bug_id)

df_ld = pandas.io.sql.read_sql(sql_ld, conn)

print df_ld.dtypes

bug_id          int64
description    object
dtype: object


/root/.virtualenvs/dup_bug_retrieval/lib/python2.7/site-packages/pandas/io/sql.py:1462: Warning: 785 line(s) were cut by GROUP_CONCAT()
  cur.execute(*args)


In [4]:
# text = re.sub("[^a-zA-Z_/\-\.]", " ", "description_text is 5-9 /etc words long.")
# print text
# text = re.sub("\.(?!((c|h|cpp|py)\s+$))", " ", text)
# print text
# print bug_id
# print ','.join(bug_id)

In [5]:
def discription_to_wordlist( raw_description ):
    # Function to convert a raw bug description to a string of words
    # 1. Remove markers
    description_text = BeautifulSoup(raw_description).get_text() 
    #
    # 2. Remove non-letters        
    # letters_only = re.sub("[^a-zA-Z_/\-\.]", " ", description_text)
    letters_only = re.sub("[^a-zA-Z\.]", " ", description_text)
    letters_only = re.sub("\.(?!((c|h|cpp|py)\s+$))", " ", letters_only)
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Stemming
    porter = PorterStemmer()
    stemmed_meaningful_words = [porter.stem_word(w) for w in meaningful_words]
    
    return stemmed_meaningful_words

In [6]:
def discription_to_string( raw_description ):
    
    return " ".join(discription_to_wordlist(raw_description))

In [7]:
print discription_to_string( "virtualization by vmware!" )

virtual vmware


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",ngram_range=(1, 2), tokenizer = None, preprocessor = None,
                             stop_words = None, lowercase = False, 
                             max_df = 0.9, min_df = 10, max_features = 100000)

desc_vectors = vectorizer.fit_transform(texts)

{u'dell': 116, u'net': 277, u'crossport': 104, u'vcenter': 460, u'type': 443, u'relat': 353, u'warn': 485, u'root': 374, u'want': 484, u'risk': 373, u'hot': 205, u'attempt': 30, u'third': 431, u'maintain': 258, u'affect': 9, u'fix': 174, u'cpd': 99, u'resum': 367, u'vcpu': 461, u're': 341, u'rc': 339, u'iso': 226, u'unabl': 445, u'keyword': 234, u'even': 157, u'explain': 163, u'new': 279, u'patch': 305, u'result': 366, u'fail': 164, u'best': 39, u'login': 252, u'diff': 128, u'child': 68, u'init': 218, u'servic': 386, u'io': 224, u'id': 211, u'sever': 388, u'make': 259, u'applic': 20, u'specif': 402, u'manual': 262, u'right': 372, u'intern': 223, u'track': 436, u'support': 416, u'avail': 33, u'call': 54, u'recommend': 348, u'exist': 161, u'vmmemctl': 475, u'time': 433, u'osp': 297, u'vmcore': 471, u'item': 228, u'team': 425, u'mt': 272, u'vmxnet': 479, u'test': 427, u'address': 7, u'wait': 483, u'instal': 221, u'askvmwar': 25, u'date': 112, u'data': 109, u'sr': 403, u'sp': 400, u'pointe

In [9]:
from sklearn import mixture
dpgmm = mixture.DPGMM(n_components=500, covariance_type='full', alpha=100, n_iter=5)
dpgmm.fit(desc_vectors.toarray())

DPGMM(alpha=100.0, covariance_type='full', init_params='wmc', min_covar=None,
   n_components=500, n_iter=5, params='wmc', random_state=None,
   thresh=None, tol=0.001, verbose=False)

In [11]:
print dpgmm.predict(desc_vectors.toarray())

[44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44

In [91]:
# df['words'] = df['short_desc'].map(discription_to_words)
# vocabulary = sorted(set(itertools.chain(*df['words'])))
# vocabulary = [(item,) for item in vocabulary]
# conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
# cur =conn.cursor()
# cur.executemany('''insert into summ_vocabulary (word) values (%s)''', vocabulary)
# conn.commit()

In [92]:
# l=['a','b','c']
# l=[(item,) for item in l]
# print l

In [93]:
# vocabulary = [(item,) for item in vocabulary]
# cur.executemany('''insert into summ_vocabulary (word) values (%s)''', vocabulary)

In [94]:
# print df['words'][0]
# itertools.chain(*['words'])
print vocabulary

[(u'.c',), (u'.cpp',), (u'.h',), (u'.pi',), (u'a',), (u'aa',), (u'aaa',), (u'aaaa',), (u'aaaaaa',), (u'aaaaaaa',), (u'aaaaaaaa',), (u'aaaaaaaaaaabaaaa',), (u'aaaabaf',), (u'aaaabb',), (u'aaaac',), (u'aaab',), (u'aaabd',), (u'aaac',), (u'aaad',), (u'aaadadc',), (u'aaae',), (u'aaafcd',), (u'aaapter',), (u'aaassssseesrerrttt',), (u'aab',), (u'aaba',), (u'aabb',), (u'aabbf',), (u'aabc',), (u'aabcc',), (u'aabd',), (u'aabf',), (u'aabramovitz',), (u'aac',), (u'aaca',), (u'aacb',), (u'aacc',), (u'aaccept',), (u'aacda',), (u'aaceb',), (u'aacf',), (u'aachba',), (u'aacmu',), (u'aacp',), (u'aacraid',), (u'aad',), (u'aadapt',), (u'aadb',), (u'aadbbf',), (u'aadc',), (u'aadd',), (u'aadda',), (u'aaddb',), (u'aaddf',), (u'aadf',), (u'aaditya',), (u'aadr',), (u'aae',), (u'aaea',), (u'aaebf',), (u'aaec',), (u'aaedcd',), (u'aaee',), (u'aaeeed',), (u'aaef',), (u'aaf',), (u'aafb',), (u'aafc',), (u'aafebd',), (u'aafef',), (u'aaff',), (u'aaffb',), (u'aag',), (u'aagarw',), (u'aagharkar',), (u'aagin',), (u'aagr

In [4]:
print df['thetext'].apply(discription_to_words)

0    panic user-level causes non-obvious failures m...
1                       bug fixed. seem happen anymore
2                                            linux-hos
3    gnats-environment-fields: host os: guest os: p...
4    gnats-environment-fields: host os: guest os: p...
5    due import siebel tickets salesforce.com, srs ...
6    server: perforce-releng.eng.vmware.com:1850 ch...
Name: thetext, dtype: object


In [6]:
print df.head(1)["thetext"]

0    A panic at user-level causes non-obvious failu...
Name: thetext, dtype: object


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 5000)
print vectorizer.stop_words

english


In [12]:
print df['thetext'][0]

A panic at user-level causes non-obvious failures modes (e.g. core dumps)
in different modules depending on the level of initialization

ASSERT(0) in various places at user-leve

